# quick start

## 处理数据

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
# `torchvision.datasets` 模块包含许多现实世界视觉数据`Dataset`，例如 CIFAR、COCO
# ([数据集列表](https://pytorch.org/vision/stable/datasets.html))。
# 在本教程中，我们使用 `FashionMNIST`数据集。每个TorchVision `Dataset`包括两个参数：`transform` 和 `target_transform`，分别用于修改样本数据和标签。
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)
# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)


In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## 创建模型

In [8]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

#Define model

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## 优化模型参数

In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [21]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}\n[{current:>5d}/{size:>5d}]")

In [22]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            test_loss /= num_batches
            correct /= size
            print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [23]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.140483
[   64/60000]
loss: 1.148296
[ 6464/60000]
loss: 0.967827
[12864/60000]
loss: 1.109643
[19264/60000]
loss: 0.986701
[25664/60000]
loss: 1.005500
[32064/60000]
loss: 1.045905
[38464/60000]
loss: 0.986280
[44864/60000]
loss: 1.027949
[51264/60000]
loss: 0.959498
[57664/60000]
Test Error: 
 Accuracy: 0.4%, Avg loss: 0.006209 

Test Error: 
 Accuracy: 0.5%, Avg loss: 0.005339 

Test Error: 
 Accuracy: 0.5%, Avg loss: 0.005917 

Test Error: 
 Accuracy: 0.4%, Avg loss: 0.006002 

Test Error: 
 Accuracy: 0.4%, Avg loss: 0.006277 

Test Error: 
 Accuracy: 0.5%, Avg loss: 0.005664 

Test Error: 
 Accuracy: 0.4%, Avg loss: 0.005936 

Test Error: 
 Accuracy: 0.4%, Avg loss: 0.006061 

Test Error: 
 Accuracy: 0.4%, Avg loss: 0.006130 

Test Error: 
 Accuracy: 0.4%, Avg loss: 0.007000 

Test Error: 
 Accuracy: 0.5%, Avg loss: 0.006584 

Test Error: 
 Accuracy: 0.4%, Avg loss: 0.006429 

Test Error: 
 Accuracy: 0.5%, Avg loss: 0.005666 

Test Er

## 保存模型

In [25]:
torch.save(model.state_dict(), "model_qucik_start.pth")
print("Saved PyTorch Model State to model_quick_start.pth")

Saved PyTorch Model State to model_quick_start.pth


## 加载预测

In [26]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model_qucik_start.pth"))

<All keys matched successfully>

In [29]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[2][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Trouser", Actual: "Ankle boot"
